`rpy2` runs a real R session "in the back room", and moves data back and forth between the foreground Python application and the R session in various ways.

You can send text directly to the R session to be evaluated.  You can also access "wrapper" objects which pull R data into Python space, and pass data from Python into R.

## Installation

### Using existing R

### Using R with (Ana)conda

In [ ]:
# Simply send text to R
import rpy2
from rpy2 import robjects  # (R = rpy2.robject.r doesn't work)
R = robjects.r
# R("install.packages(c('ggplot2', 'spsurvey'))")
# R windows for selecting mirror and downloading progress pop up
R("library('spsurvey')")

In [ ]:
# Or, if you wanted to manipulate R package installation with Python
import rpy2.robjects.packages as rpackages  # access R packages
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
required = ['ggplot2', 'spsurvey']  # a Python variable (list)
for req in required:
    if not rpackages.isinstalled(req):
        utils.install_packages(req)
# this seems convoluted, but installs packages only if needed, so it's quick,
# and keeps everything on the Python side.
for req in required:
    R.library(req)

**Note:** `R...install.packages...` but `utils.install_packages` - `_` not `.` in Python.

In [ ]:
# setup automatic pandas <-> R dataframe conversions
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [ ]:
import os
R.setwd("u:/repo/PyR")
os.chdir("u:/repo/PyR")

In [ ]:
%matplotlib inline
import geopandas
import matplotlib.pyplot as plt

lakes = geopandas.read_file("Shapes/Great_Lakes_5.shp")
lakes.plot()

In [ ]:
lakes.head()

### spsurvey: Spatial Survey Design and Analysis

"These functions provide procedures for selecting sites for spatial surveys using spatially balanced algorithms applied to discrete points, linear networks, or polygons. The probability survey designs available include independent random samples, stratified random samples, and unequal probability random samples (categorical or probability proportional to size)."

Original R code
```R
# Create the design list
Stratdsgn <- list(
  "Lake Superior"=list(panel=c(PanelOne=50), seltype="Equal", over=20),
  "Lake Michigan"=list(panel=c(PanelOne=30), seltype="Equal", over=15),
  "Lake Huron"=list(panel=c(PanelOne=30), seltype="Equal", over=15),
  "Lake Erie"=list(panel=c(PanelOne=20), seltype="Equal", over=10),
  "Lake Ontario"=list(panel=c(PanelOne=20), seltype="Equal", over=10))
```

This uses R `list()`s for a collection of named attributes at two levels, but then switches to using an R vector (`c()`) for a collection of one named attribute.  Starting from scratch I would probably either put this data in a separate file (.csv) or use a `namedtuple` to make it less cluttered.  But translating directly to Python:

In [ ]:
Stratdsgn = {
  'Lake Superior': {'panel': {'PanelOne': 50}, 'seltype': "Equal", 'over': 20},
  'Lake Michigan': {'panel': {'PanelOne': 30}, 'seltype': "Equal", 'over': 15},
  'Lake Huron': {'panel': {'PanelOne': 30}, 'seltype': "Equal", 'over': 15},
  'Lake Erie': {'panel': {'PanelOne': 20}, 'seltype': "Equal", 'over': 10},
  'Lake Ontario': {'panel': {'PanelOne': 20}, 'seltype': "Equal", 'over': 10},
}

We need to convert this to R objects to call the R function.  `R.list`, aka `rpy2.robjects.r['list']` is the R `list()` function, which takes a series of `keyword=value` pairs.  In Python, keywords can't be expressions, like the string expression, `"Lake Superior"`, so you could write:

In [ ]:
R_Stratdsgn = R.list(
  Lake_Superior=R.list(panel=R.c(PanelOne=50), seltype="Equal", over=20),
  Lake_Michigan=R.list(panel=R.c(PanelOne=30), seltype="Equal", over=15),
  Lake_Huron=R.list(panel=R.c(PanelOne=30), seltype="Equal", over=15),
  Lake_Erie=R.list(panel=R.c(PanelOne=20), seltype="Equal", over=10),
  Lake_Ontario=R.list(panel=R.c(PanelOne=20), seltype="Equal", over=10),
)

Or you could use Python's `**<dict>` operator, which converts the `<dict>` into a series of `keyword=value` pairs:

In [ ]:
R_Stratdsgn = R.list(**{
  'Lake Superior': R.list(panel=R.c(PanelOne=50), seltype="Equal", over=20),
  'Lake Michigan': R.list(panel=R.c(PanelOne=30), seltype="Equal", over=15),
  'Lake Huron': R.list(panel=R.c(PanelOne=30), seltype="Equal", over=15),
  'Lake Erie': R.list(panel=R.c(PanelOne=20), seltype="Equal", over=10),
  'Lake Ontario': R.list(panel=R.c(PanelOne=20), seltype="Equal", over=10),
})

But if you're manipulating data in Python and only want the R version to call an R function, maybe convert the Python data structure to R objects like this:

In [ ]:
R_Stratdsgn = R.list()  # top level list
for lake, setup in Stratdsgn.items():
    R_Stratdsgn = R.append(
        R_Stratdsgn, 
        R.list(**{lake: R.list(  # lists within list
            panel=R.c(**setup['panel']),  # R character vector
            seltype=setup['seltype'],
            over=setup['over'],
        )})
    )

Original R code
```R
att <- read.dbf("Great_Lakes_5")
set.seed(4447864) #This allows you to re-create the points if needed

Stratsites <- grts(design=Stratdsgn,
 DesignID="Stratified",
 type.frame="area",
 src.frame="shapefile",
 in.shape="Great_Lakes_5",
 att.frame=att,
 stratum="PNAME",
 shapefile=TRUE,
 prjfilename="Great_Lakes_5",
 out.shape="Stratified_Equal")
```

In [ ]:
R("set.seed(4447864)")  # This allows you to re-create the points if needed
# OR
base = rpackages.importr('base')
base.set_seed(4447864)
# OR
R("set.seed(%s)" % 4447864)  # generate text and pass it to R
# OR
R['set.seed'](4447864)

In [ ]:
Stratsites = R['grts'](**{
    'design': R_Stratdsgn,
    'DesignID': "Stratified",
    'type.frame': "area",
    'src.frame': "shapefile",
    'in.shape': "Shapes\\Great_Lakes_5",
    'att.frame': lakes.drop(columns='geometry'),
    'stratum': "PNAME",
    'shapefile': True,
    'prjfilename': "Shapes\\Great_Lakes_5",
    'out.shape': "Stratified_Equal",
})

In [ ]:
pts = geopandas.read_file("Stratified_Equal.shp")
pts.plot()

In [ ]:
pts.head()

In [ ]:
Stratsites
dir(Stratsites)
Stratsites.slots
dir(Stratsites.slots)
Stratsites.slots.keys
Stratsites.slots.keys()
list(Stratsites.slots.keys())
list(Stratsites.slotnames())
# ...
list(Stratsites.slots.items())
list(Stratsites.list_attrs())
Stratsites.do_slot('coords')
d = dict(Stratsites.slots.items())
list(d)
d['design']
d['data']


## Exchanging data frames

In [ ]:
R.library('datasets')
R.data('iris')
type(R.iris)

In [ ]:
R.iris.head()

In [ ]:
R.iris['color'] = 'red'
R.iris.columns

In [ ]:
iris = R.iris
iris['color'] = 'red'
iris.columns

In [ ]:
from rpy2.robjects.packages import importr
grdevices = importr('grDevices')

iris.loc[iris.Species == 'setosa', 'color'] = 'blue'
R.head(iris)
iris.head()

grdevices.png(file="test.png", width=512, height=512)
R.plot(x=iris['Sepal.Length'], y=iris['Sepal.Width'])
grdevices.dev_off()

R.assign('iris', iris)


In [ ]:
def x(**kwargs):
    print("Args:")
    for k, v in kwargs.items():
        print(k, v)

x(a=2, b=3)
x(c=1, **{'a':2, "Bee's": 3})